In [2]:
import re
import datetime
import pandas as pd
import json
from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from pandasgui import show
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

In [3]:
client = PIWebApiClient("https://piwebapi.cammesa.com/piwebapi",
                            useKerberos=False, verifySsl=True)

In [4]:

verano1_start = datetime.date(2021, 12, 1)
verano1_end = datetime.date(2022, 3, 1)

verano2_start = datetime.date(2022, 12, 1)
verano2_end = datetime.date(2023, 3, 1)

verano3_start = datetime.date(2023, 12, 1)
verano3_end = datetime.date(2024, 2, 28)

fecha_start = datetime.date(2024, 3, 15)
fecha_end = datetime.date(2024, 3, 15)

In [5]:
periodos = [
    #(verano1_start, verano1_end),
    #(verano2_start, verano2_end),
    #(verano3_start, verano3_end)
    (fecha_start, fecha_end),
]

In [6]:
df_paths = pd.read_csv('paths_reducido.csv')
paths = df_paths['Paths'].tolist()

In [8]:
from osisoft.pidevclub.piwebapi.rest import ApiException
df_nea = pd.DataFrame()

for periodo in periodos:
    start_datetime = datetime.datetime.combine(periodo[0], datetime.time(14,0,0))
    end_datetime = datetime.datetime.combine(periodo[1], datetime.time(15,0,0))

    for k, pt in enumerate(paths):
        try:
            df_interpolate = client.data.get_interpolated_values(
                path=pt,
                start_time=start_datetime,
                end_time=end_datetime,
                interval="1m"
            )

            if df_interpolate.empty or not all(col in df_interpolate.columns for col in ['Timestamp', 'Value']):
                print(f"Tag not found or data is missing for path: {pt}. Skipping...")
                continue 

            df_interpolate = df_interpolate[['Timestamp', 'Value']]
            df_interpolate.set_index('Timestamp', inplace=True)

            column_name = pt.replace('pi:\\\\CAMHIS01\\', '')

            df_interpolate.rename(columns={'Value': column_name}, inplace=True)

            if df_nea.empty:
                df_nea = df_interpolate
            else:
                df_nea = pd.concat([df_nea, df_interpolate], axis=1)

        except ApiException as e:
            if e.status == 404:
                print(f"Tag not found for path: {pt}. Skipping...")
                continue  # Skip to the next iteration
            else:
                # Handle other ApiException or re-raise the exception if needed
                raise




In [9]:
df_nea.head()

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,ALLEN/132/G.ROCA/I.AV,ALLEN/132/G.ROCA/P.AV,ALLEN/132/G.ROCA/Q.AV,ALLEN/132/G.ROCA/V.AV,...,S.ISIDRO/132/TRAFO1/V.AV,SGO.ESTE/13/ROCA/I.AV,SMARTNEA/13/TRAFO2/I.AV,SMARTNEA/13/TRAFO2/P.AV,SMARTNEA/13/TRAFO2/Q.AV,SMARTNEA/13/TRAFO2/V.AV,SMARTNEA/132/E.COLOR/I.AV,SMARTNEA/132/E.COLOR/P.AV,SMARTNEA/132/E.COLOR/Q.AV,SMARTNEA/132/E.COLOR/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-03-15T17:00:00Z,2160.92944,33.059166,28.859817,234.072700,49.059204,-25.824749,150.799988,-34.559998,-3.63,131.239990,...,137.915375,210.201416,153.900000,-3.52836,-0.58806,13.424400,0.0,0.0,0.0,124.399994
2024-03-15T17:01:00Z,2154.85083,33.059166,28.810990,255.020600,50.685204,-24.748074,151.600000,-35.030000,-3.49,131.299988,...,137.915375,210.952057,153.900000,-3.52836,-0.58806,13.424400,0.0,0.0,0.0,124.000000
2024-03-15T17:02:00Z,2164.41846,32.863846,29.103966,246.399719,48.795525,-25.846722,151.199982,-34.850000,-3.85,131.169983,...,137.915375,208.700134,154.799988,-3.52836,-0.58806,13.424400,0.0,0.0,0.0,124.000000
2024-03-15T17:03:00Z,2067.73975,32.863846,28.957478,244.385500,49.344850,-26.044480,154.699982,-35.400000,-3.53,131.239990,...,139.462280,207.949500,126.899994,-2.92248,-0.58806,13.424400,0.0,0.0,0.0,97.000000
2024-03-15T17:04:00Z,1873.37400,32.863846,28.664501,234.838100,48.180283,-26.088425,155.799988,-35.839996,-3.25,131.250000,...,140.429108,207.198853,126.899994,-2.92248,-0.58806,13.793999,0.0,0.0,0.0,140.900000


In [10]:
df_nea_hora = df_nea.copy()
df_nea_hora.index = pd.to_datetime(df_nea_hora.index).tz_convert('America/Argentina/Buenos_Aires').tz_localize(None)

df_nea_hora

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,ALLEN/132/G.ROCA/I.AV,ALLEN/132/G.ROCA/P.AV,ALLEN/132/G.ROCA/Q.AV,ALLEN/132/G.ROCA/V.AV,...,S.ISIDRO/132/TRAFO1/V.AV,SGO.ESTE/13/ROCA/I.AV,SMARTNEA/13/TRAFO2/I.AV,SMARTNEA/13/TRAFO2/P.AV,SMARTNEA/13/TRAFO2/Q.AV,SMARTNEA/13/TRAFO2/V.AV,SMARTNEA/132/E.COLOR/I.AV,SMARTNEA/132/E.COLOR/P.AV,SMARTNEA/132/E.COLOR/Q.AV,SMARTNEA/132/E.COLOR/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-03-15 14:00:00,2160.92944,33.059166,28.859817,234.072700,49.059204,-25.824749,150.799988,-34.559998,-3.63,131.239990,...,137.915375,210.201416,153.900000,-3.52836,-0.58806,13.424400,0.0,0.0,0.0,124.399994
2024-03-15 14:01:00,2154.85083,33.059166,28.810990,255.020600,50.685204,-24.748074,151.600000,-35.030000,-3.49,131.299988,...,137.915375,210.952057,153.900000,-3.52836,-0.58806,13.424400,0.0,0.0,0.0,124.000000
2024-03-15 14:02:00,2164.41846,32.863846,29.103966,246.399719,48.795525,-25.846722,151.199982,-34.850000,-3.85,131.169983,...,137.915375,208.700134,154.799988,-3.52836,-0.58806,13.424400,0.0,0.0,0.0,124.000000
2024-03-15 14:03:00,2067.73975,32.863846,28.957478,244.385500,49.344850,-26.044480,154.699982,-35.400000,-3.53,131.239990,...,139.462280,207.949500,126.899994,-2.92248,-0.58806,13.424400,0.0,0.0,0.0,97.000000
2024-03-15 14:04:00,1873.37400,32.863846,28.664501,234.838100,48.180283,-26.088425,155.799988,-35.839996,-3.25,131.250000,...,140.429108,207.198853,126.899994,-2.92248,-0.58806,13.793999,0.0,0.0,0.0,140.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-15 14:56:00,1686.87463,31.838425,28.176205,235.522919,49.322876,-26.835506,154.900000,-35.540000,-4.04,130.930000,...,136.755157,209.450775,113.849991,-1.10484,-2.74428,13.859999,0.0,0.0,0.0,127.300000
2024-03-15 14:57:00,1706.48608,31.838425,28.176205,233.750400,46.664146,-27.165100,154.900000,-35.369995,-3.87,130.810000,...,136.755157,207.949500,117.000000,-1.46124,-2.40570,13.701599,0.0,0.0,0.0,126.800000
2024-03-15 14:58:00,1730.98900,31.984913,28.176205,240.679321,47.740820,-27.165100,155.400000,-35.600000,-3.70,130.930000,...,136.755157,207.198853,117.000000,-1.46124,-2.40570,13.595999,0.0,0.0,0.0,125.300000


In [81]:
df_nea_hora.to_pickle('colapso_15032024.pkl')

In [22]:
df_nea.to_csv('colapsos_reducido_1.csv', index=True)